# Share Analysis

In [ ]:
# Solve SynchronousOnlyOperation error
# https://stackoverflow.com/questions/61926359/django-synchronousonlyoperation-you-cannot-call-this-from-an-async-context-u

import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from voices.website import models

### Extract data from db and convert it in a seaborn compatible data source

In [ ]:
shares = models.Share.objects.order_by("timestamp").all()
data = {
    "timestamp": [s.timestamp for s in shares],
    "latitude": [s.latitude for s in shares],
    "longitude": [s.longitude for s in shares],
    "day": [s.timestamp.date() for s in shares],
}

In [ ]:
%matplotlib ipympl
sns.scatterplot(data=data, x="latitude", y="longitude", hue="day")

In [ ]:
# Decomment to custoomize and update data if in ipympl mode
# sns.scatterplot(data=data, x="latitude", y="longitude", hue="day")